![lyon2 geonum](https://perso.liris.cnrs.fr/lmoncla/GEONUM/fig/logos.png)

# 2A3 – Modélisation et structuration de données géographiques et applications SGBD spatiaux

## Tutoriel : Analyse des données des disponibilités des stations Vélo'v de la Métropole de Lyon

# Partie 3 : Un peu de science des données


L'objectif de cette partie du tutoriel est de continuer l'analyse des données velo'v et d'expérimenter des méthodes d'intelligence artificielle pour le clustering des données mais également d'ajouter une couche de données météo pour faire de la prédiction des disponibilités vélo'v.

Les objectifs de cette partie sont les suivants : 

* Entraîner un modèle d'apprentissage non supervisée (clustering) pour l'analyse des données.
* Récupérer le jeu de données météo et l'associer à celui des velo'v.
* Visualiser les données selon cette nouvelle couche météo.
* Entraîner un modèle de prédiction de la demande des vélo'v.


## 1. Configurer l'environnement

### 1.2 Télécharger les fichiers et installer les librairies (seulement pour Google Colab)

* Si vous avez déjà configuré votre environnement, soit avec conda, soit avec pip (voir le fichier [README.md](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/README.md)), vous pouvez ignorer la section suivante et passer directement à la section 1.2.
* Si vous exécutez ce notebook depuis Google Colab, vous devez exécuter les cellules suivantes :

In [ ]:
! git clone https://github.com/ludovicmoncla/master-geonum-tutorials.git

In [ ]:
! pip install -r master-geonum-tutorials/requirements.txt

### 1.2 Importer les librairies

In [ ]:
import pandas as pd
import numpy as np

import folium
import plotly
import plotly.express as px
import geopandas

import sklearn.cluster

## 2. Récupération des jeux de données

Comme pour la Partie 1, l'ensemble des données utilisées dans ce tutoriel est disponible à cette adresse : 
https://perso.liris.cnrs.fr/lmoncla/GEONUM/

* Télécharger les fichiers contenant les données
1. data-stations.zip
2. data-bikes-2.zip
3. data-weather-lyon.csv

Les 2 zip contiennent chacun un fichier CSV contenant respectivement la liste des stations vélov (et leur localisation) et la liste des disponibilités de chaque station par tranche de 30 minutes.


In [ ]:
## On commence par créer un dossier dans lequel on va télécharger les données
## Peut être fait directement depuis l'explorateur de fichiers
!mkdir data

In [ ]:
## on se déplace dans le nouveau dossier
%cd data

In [ ]:
## On télécharge l'archive contenant la liste des stations
!wget https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-stations.zip
    
## On télécharge l'archive contenant la liste des disponibilité des stations par tranche de 5 minutes
!wget https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-bikes-2.zip


### 2.1. Chargement des données

Nous chargeons les mêmes jeux de données que dans le tutoriel précédent :

1. les stations vélo'v (id station, latitude, longitude),
2. leurs historiques (id station, année, mois, jour, heure, minute, date, vélos disponibles, places disponibles).


In [ ]:
## On charge les données des stations dans un dataframe
df_stations = *****

## On crée maintenant le dataframe avec les données d'historique
df_bikes = *****

In [ ]:
## On affiche les premières lignes
*****

In [ ]:
# Réduction de la taille en mémoire

## on transforme le type des colonnes en entier ou float lorsque cela est nécessaire
df_bikes['time'] = pd.to_datetime(df_bikes['time']) 
df_bikes[['year', 'daily_departure', 'daily_arrival']] = df_bikes[['year', 'daily_departure', 'daily_arrival']].astype('int16')
df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']] = df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']].astype('int8')


## 3. Clustering

Notre objectif dans cette partie va être d'identifier des groupes de stations "similaires". Pour cela, nous allons appliquer des méthodes d'apprentissage non supervisées : clustering.
L'obectif n'est pas de regrouper les stations par proximité spatial mais par une similarité calculée à partir des données d'historique de l'utilisation des stations.

### 3.1 Préparation des données

Dans un premier temps nous devons nous assurer que toutes les stations sont comparables. On s'intèresse donc à savoir si elles ont toutes le même nombre de données. Le manque de données peut provenir de bugs à différents niveaux du processus de mise à disposition des données mais cela peut également être dû à des stations qui auraient fermés en cours d'année.

Afin de vérifier si toutes les stations ont le même nombre de données, nous pouvons regrouper les lignes du dataframe en fonction du nom de la station puis afficher la taille de chaque groupe.

In [ ]:
# On regroupe les lignes du dataframe par station
g = *****

# On affiche la taille de chaque groupe
print(g.size())

Toutes les stations ne s'affichent pas mais on peut déjà observer qu'une station à moins de données que les autres. Nous devons déterminer combien de stations sont différentes pour les supprimer.
Utiliser la fonction [plot()](https://www.geeksforgeeks.org/plot-the-size-of-each-group-in-a-groupby-object-in-pandas/) pour afficher un graphique de la taille (nombre de données) de chaque station.

In [ ]:
# On affiche les tailles de chaque groupe sur un graphique
*****

On remarque que 6 stations ont nettement moins de données que les autres. Pour le reste des traitements nous allons donc supprimer ces 6 stations du jeu de données.

Identifier le nom de ces 6 stations en utilisant les fonctions count() et [nsmallest()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nsmallest.html).

In [ ]:
# On affiche la liste des 6 stations ayant le moins de données
list_to_drop = *****
list_to_drop

In [ ]:
# On vérifie la taille du dataframe avant la suppression des lignes des stations concernées
df_bikes.shape

In [ ]:
# Supprimer les 6 stations du dataframe df_bikes

df_bikes.drop(*****].index, inplace=True)


In [ ]:
# On vérifie la taille du dataframe après la suppression
df_bikes.shape

Afin de pouvoir regrouper les stations par similarité, nous allons ajouter quelques variables (colonnes). En particulier, nous allons nous intéresser au nombre de départs (et d'arrivées) normalisé en fonction du jour de la semaine. 


- Créer une nouvelle colonne `day_of_week` en utilisant la méthode [day_name()](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.day_name.html)



In [ ]:
# crée une nouvelle colonne avec le nom du jour de la semaine
df_bikes['day_of_week'] = df_bikes['time'].dt.day_name()

In [ ]:
df_bikes.head()

On souhaite maintenant ajouter des colonnes avec les valeurs journalière des départs et arrivées

In [ ]:
# On calcule les moyennes des arrivées et des départs (journalier) par station et par jour de la semaine
arrivals = *****
departures = *****


In [ ]:
arrivals = arrivals.unstack(level=1) # transforme les lignes en colonnes
arrivals = arrivals.fillna(0) # on remplace les valeurs vide null
arrivals

In [ ]:
departures = departures.unstack(level=1) # transforme les lignes en colonnes
departures = departures.fillna(0) # on remplace les valeurs null
departures

In [ ]:
# On combine ces deux jeux de données en un seul qui servira de jeu d'entrainement pour l'algorithme de clustering

df_data = departures.merge(arrivals, how='inner', on=['id_velov'])
df_data = df_data.fillna(0)
df_data.head()

In [ ]:
# On vérifie s'il existe des valeurs infinies (non compatible avec l'algo de clustering)
np.any(np.isfinite(df_data))

In [ ]:
# On remplace ces valeur par la valeur 1
df_data.replace([np.inf, -np.inf], 1, inplace=True)

In [ ]:
df_data.head()

### 3.2 Entraînement et utilisation du modèle de clustering



Les méthodes d'apprentissage automatique sont déjà implémentées au sein de la librairie [Scikit-Learn](https://scikit-learn.org/stable/). Cette librairie regroupe un grand nombre d'algorithmes d'apprentissage non supervisée (clustering) et supervisée (classification et regression).

La méthode des [k-moyennes](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) ou Kmeans est un algorithme de clustering très utilisé. Son principe est simple regrouper les données en k clusters homogènes et compacts. Afin de créer des clusters homogènes l'algorithme se base sur un calcul de distance entre les données et le centroid des différents clusters. Ces centroids étant recalculer à chaque fois qu'une nouvelle données est ajoutée au cluster. 

Notre objectif ici, est de déterminer si les stations peuvent être regroupées en 2 clusters distincts en se basant sur la similarité de leur historique d'utilisation.



In [ ]:
# déclaration du modèle
model = *****

# entrainement du modèle
*****

# utilisation du modèle pour associer un numéro de cluster à chaque ligne du jeu de données
df_data["cluster"] = *****

In [ ]:
df_data.head()

Pour pouvoir afficher ces clusters sur une carte il faut ajouter les coordonées latitude/longitude des stations à notre dataframe.

In [ ]:
df_data = *****
df_data.head()

### 3.3 Représentation cartographique des clusters

In [ ]:
# On transforme le dataframe en geodataframe
*****

In [ ]:
## On affiche directement les données du geodataframe sur une carte 
## avec la méthode scatter_mapbox() de la librairie plotly.express:
fig = *****

## On supprime les marges autour de la carte
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update(layout_coloraxis_showscale=False) # supprime la colorbar

## On affiche la carte
fig.show()

On observe que les 2 clusters sont également distincts géographiquement. On remarque un cluster situé dans le centre et le deuxième situé plutôt en périphérie.

## 4. Ajout d'une couche de données supplémentaire : la météo


### 4.1 Chargement des données


Le fichier `data-weather-lyon.csv` contient les données météo par heure pour l'année 2021 pour Lyon.

Le jeu de données météo correspond au dataset `NASA/POWER CERES/MERRA2 Native Resolution Hourly Data` récupéré grâce à l'API du site POWER Project de la NASA : https://power.larc.nasa.gov/.




In [ ]:
## On télécharge l'archive contenant les données météo
!wget https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-weather-lyon.csv

In [ ]:
## On charge les données météo dans un dataframe
df_weather = pd.read_csv('data-weather-lyon.csv')

In [ ]:
## On affiche les premières lignes
df_weather.head()

### 4.2. Premier apercu des données météo

Les colonnes du jeu de données météo sont les suivantes : 

- WD10M           MERRA-2 Wind Direction at 10 Meters (Degrees) 
- T2M             MERRA-2 Temperature at 2 Meters (C) 
- RH2M            MERRA-2 Relative Humidity at 2 Meters (%) 
- QV2M            MERRA-2 Specific Humidity at 2 Meters (g/kg) 
- T2MDEW          MERRA-2 Dew/Frost Point at 2 Meters (C) 
- U10M            MERRA-2 Eastward Wind at 10 Meters (m/s) 
- PS              MERRA-2 Surface Pressure (kPa) 
- T2MWET          MERRA-2 Wet Bulb Temperature at 2 Meters (C) 
- WS10M           MERRA-2 Wind Speed at 10 Meters (m/s) 
- V10M            MERRA-2 Northward Wind at 10 Meters (m/s) 
- PRECTOTCORR     MERRA-2 Precipitation Corrected (mm/hour) 


Utiliser la méthode [drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) pour supprimer les colonnes que l'on ne souhaite pas conserver.

In [ ]:
## On supprime les colonnes inutiles
## On ne souhaite conserver que les colonnes température, vitesse du vent et pluviométrie.

df_weather = *****

df_weather.head()

 Utiliser la méthode [rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) pour renommer les colonnes restantes afin de pouvoir faire une jointure avec les données de disponibilité vélo'v.

In [ ]:
## On renomme les colonnes pour ensuite faire une jointure de ce dataframe avec celui des données vélo'v

df_weather = *****

df_weather.head()


In [ ]:
## On affiche les moyennes des températures, précipitation et vent par mois sur 3 graphiques différents

fig,(ax1,ax2,ax3)= plt.subplots(nrows=3)
fig.set_size_inches(12,20)

monthAggregated = pd.DataFrame(*****).reset_index()
monthSorted = monthAggregated.sort_values(by='temperature', ascending = False) 
sn.barplot(data=monthSorted, x = 'month', y = 'temperature', ax=ax1)
ax1.set(xlabel='Month', ylabel='Average Temperature', title='Average Temperature by Month') 

monthAggregated = pd.DataFrame(*****).reset_index()
monthSorted = monthAggregated.sort_values(by='precipitation', ascending = False) 
sn.barplot(data=monthSorted, x = 'month', y = 'precipitation', ax=ax2)
ax2.set(xlabel='Month', ylabel='Average Precipitation', title='Average Precipitation by Month') 

monthAggregated = pd.DataFrame(*****).reset_index()
monthSorted = monthAggregated.sort_values(by='vent', ascending = False) 
sn.barplot(data=monthSorted, x = 'month', y = 'vent', ax=ax3)
ax3.set(xlabel='Month', ylabel='Average Wind Speed', title='Average Wind Speed by Month') 

Utiliser la méthode [merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) pour combiner les données de disponibilité des velo'v avec les données méteo.

In [ ]:
## On ajoute les données météo dans le dataframe qui contient les données de disponibilité
df_bikes = *****
df_bikes.head()

## 5. Entraînement d'un modèle de prédiction

### 5.1 Import de la librairie (scikit-learn)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

### 5.2 Préparation des données pour l'apprentissage

In [ ]:
## On fait une copie de notre DataFrame, pour pouvoir revenir aux données initiales si besoin
df_data = df_bikes.copy()

In [ ]:
# On sépare le jeu de données en entrées et sorties pour l'apprentissage

# les inputs correspondent aux données fournis au modèle pour pouvoir apprendre. 
inputs = *****

# les labels correspondent aux valeurs que l'on souhaite prédire et donc que le modèle doit apprendre
labels = *****



La plupart des méthodes d'apprentissage ne peuvent utiliser que des variables numériques. Il faut donc transformer les variables catégorielles telle que `id_velov` ou `day_of_week` afin qu'elles puissent servir lors de l'apprentissage. 

Pour cela on utilise la fonction [LabelEncoder()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) de la librairie scikit-learn. Elle permet de transformer des valeurs de type chaine de caractère en numérique par association. Par exemple : lundi -> 0, mardi -> 1, etc...

In [ ]:
# On transforme les variables catégorielles en variables numériques
l1_encoder = LabelEncoder()
l2_encoder = LabelEncoder()

# on entraîne l'encoder pour qu'il ai vu toutes les valeurs possibles
l1_encoder.fit(df_stations.id_velov)

# on transforme la colonne id_velov
inputs['id_velov'] = l1_encoder.transform(inputs['id_velov'])

l2_encoder.fit(inputs['day_of_week'])
inputs['day_of_week'] = l2_encoder.transform(inputs['day_of_week'])

inputs.head()

Une fois que les données sont prêtes et que les variables sont au bon format. Il faut maintenant séparer le jeu de données pour utiliser une partie des données pour l'entraînement et une autre partie pour l'évaluation. 

Pour cela on utilise la méthode [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) qui permet de séparer le jeu de données en 2 de manière aléatoire pour que les jeux d'entrainement et d'évaluation aient les mêmes caractéristiques.

In [ ]:
# On sépare le jeu de données en 2 : un jeu d'entraînement et un jeu de test
x_train, x_test, y_train, y_test = *****


### 5.3 Entraînement supervisé des modèles

L'ensemble des algoritmes d'apprentissage supervisée implémentés dans la librairie Scikit-Learn sont disponibles ici : https://scikit-learn.org/stable/supervised_learning.html

Dans notre cas, nous souhaitons prédire des valeurs continues par opposition aux valeurs discrètes. Nous allons donc utiliser des modèles de régression et non pas des modèles de classification.

Je vous propose de tester et comparer 2 méthodes : [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) et [RandomForestRegressor](https://scikit-learn.org/0.15/modules/generated/sklearn.ensemble.RandomForestRegressor.html).

#### 5.3.1 Linear Regression

In [ ]:
# On déclare un modèle de type RadomForestRegressor et on l'entraîne sur le jeu d'entraînement
clf_lr = *****

*****

In [ ]:
# On utilise le modèle pour faire la prédiction sur le jeu de test
predictions_lr = *****

#### 5.3.2 Random Forest Regressor

In [ ]:
# On déclare un modèle de type RadomForestRegressor et on l'entraîne sur le jeu d'entraînement
clf_rf = *****
*****

# !! Cette cellule peut mettre plusieurs minutes à s'executer (entre 5 et 10 min) !! #

In [ ]:
# On utilise le modèle pour faire la prédiction sur le jeu de test
predictions_rf = *****

### 5.4 Evaluation et comparaison

Afin d'évaluer nos modèles de prédiction, on utilise la mesure [MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) (Mean Absolute Error) : 
$$\frac{\sum_{i=1}^n \left\lvert \hat{y}_i - y_i \right\rvert}{n}$$ 
qui permet de mesurer l'écart moyen absolu entre les prédictions et les valeurs à prédire. Plus cette valeur est faible, meilleur est notre modèle.




In [ ]:
# On calcule l'erreur moyenne en comparant les prédictions avec les valeurs qu'il fallait prédire

mae_lr = *****

print('linear regression mae : ', mae_lr)

In [ ]:
# On calcule l'erreur moyenne en comparant les prédictions avec les valeurs qu'il fallait prédire

mae_rf = *****

print('random forest regression mae : ', mae_rf)

On observe que l'erreur moyenne est nettement plus faible pour le modèle random forest ! En moyenne l'écart entre la prédiction et la réalité est de 1. Ce qui parait plutôt satisfaisant pour notre tâche. Dans le cas de l'utilisation de ce modèle dans une application, il faudrait prévenir l'utilisateur que la prédiction est en moyenne correcte à plus ou moins 1 vélo !

On souhaite maintenant afficher une évaluation sous forme de graphique. Le jeu de test contenant plus d'un million de lignes, on isole au préalable un échantillon de 100 données.

In [ ]:
# On regroupe les valeurs à prédire et les prédictions au sein d'un même tableau
t = np.stack((y_test, predictions_rf, predictions_lr), axis=0)

In [ ]:
# On vérifie la taille du tableau
t.shape

In [ ]:
# On affiche un aperçu des valeurs
t

In [ ]:
# On sélectionne un échantillon de 100 prédictions pour afficher la comparaison

idx = np.random.randint(t.shape[1], size=100)
sample = t[:,idx]

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Comparaison des prédictions")
plt.xlabel("Echantillon")
plt.ylabel("Vélos disponibles")
plt.plot(range(100), sample[0], color ="green", label="Vérité")
plt.plot(range(100), sample[1], color ="blue", label="Random Forest")
plt.plot(range(100), sample[2], color ="red", label="Linear Regression")
plt.legend()
plt.show()

On souhaite maintenant afficher le même type de graphique mais au lieu d'afficher la prédiction on veut afficher l'erreur.

In [ ]:
err_rf = abs(sample[0]-sample[1])
err_lr = abs(sample[0]-sample[2])

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Comparaison des prédictions")
plt.xlabel("Echantillon")
plt.ylabel("Erreur")
plt.plot(range(100), err_rf, color ="blue", label="Random Forest")
plt.plot(range(100), err_lr, color ="red", label="Linear Regression")
plt.legend()
plt.show()

### 5.4 Utilisation du modèle

La météo a-t-elle un impact sur la prédiction ?

Pour le savoir, nous pouvons faire une prédiction de la disponibilité à une station pour une certaine date en faisant varier la météo et observer s'il y a des différences entre les prédictions.

Le modèle prend en entrée 10 paramètres :
- id_velov
- year
- month
- day
- hour
- minute
- day_of_week
- temperature
- vent
- precipitation

On se propose de faire une prédiction pour samedi prochain à 8h à la station 3094 (Gare Part-Dieu). 
On fait la prédiction pour 2 cas différents : beau et mauvais temps
1. Beau temps : température = 20, vent = 1, précipitations = 0
2. Mauvais temps : température = 8, vent = 3, précipitations = 0.10

In [ ]:

# encoder le nom de la station
st = l1_encoder.transform(['velov-3094'])

# encoder le jour de la semaine
dn = l2_encoder.transform(['Saturday'])

d_nice = [[st, 2022, 2, 18, 8, 0, dn, 20, 1, 0]]
d_bad = [[st, 2022, 2, 18, 8, 0, dn, 8, 3, 0.10]]

# on calcule les prédictions
p_nice = *****
p_bad = *****


In [ ]:
print(p_nice)
print(p_bad)

On remarque donc que la météo a bien un impact sur la prédiction. Il y aura plus de vélos disponibles en cas de mauvais temps que de beau temps !

## Exercices

En reprenant le code des séances précédentes proposer une carte de la disponibilité des vélo'v pour l'ensemble des stations basée sur les prédictions de samedi prochain à 8h.

In [ ]:
# A COMPLETER





Proposer une carte permettant de visualiser la différence entre beau et mauvais temps pour la même date.

In [ ]:
# A COMPLETER

